In [1]:
from peft import IA3Model, IA3Config, get_peft_model
from transformers import AutoTokenizer, AutoModelForCausalLM
import gc
import torch
from tqdm import tqdm
import numpy as np
import json
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from auto_gptq import AutoGPTQForCausalLM

In [4]:
BASE_MODEL = '/home/dzigen/Desktop/ITMO/sem1/DLtech/dl_tech_learn/lab3/pretrained_models/ruGPT3_5_13B_8bit'
OUTPUT_DIR = './'
TRAIN_FILE = '/home/dzigen/Desktop/ITMO/sem1/DLtech/dl_tech_learn/lab3/data/train_part.csv'
EVAL_FILE = '/home/dzigen/Desktop/ITMO/sem1/DLtech/dl_tech_learn/lab3/data/test_part.csv'
LOG_DIR = OUTPUT_DIR + 'logs/'
ADAPTER_DIR = LOG_DIR + 'ai3_gpt3_5_8bit_adapter'

DEVICE = 'cpu'
BATCH_SIZE = 1
EPOCHS = 12
LR = 3e-4

print("BATCH_SIZE: ", BATCH_SIZE)
print("LR: ", LR)
print("EPOCHS: ", EPOCHS)
print("DEVICE: ", DEVICE)

BATCH_SIZE:  1
LR:  0.0003
EPOCHS:  12
DEVICE:  cpu


In [7]:
model = AutoGPTQForCausalLM.from_quantized(BASE_MODEL, device="cpu")

ValueError: QuantLinear() does not have a parameter or a buffer named weight.

In [4]:
TOKENIZER = AutoTokenizer.from_pretrained(BASE_MODEL)

In [3]:
MODEL = AutoModelForCausalLM.from_pretrained(BASE_MODEL, load_in_8bit=True, 
                                             use_safetensors=True, device_map='cpu')

Some weights of the model checkpoint at /home/dzigen/Desktop/ITMO/sem1/DLtech/dl_tech_learn/lab3/pretrained_models/ruGPT3_5_13B_8bit were not used when initializing GPT2LMHeadModel: ['transformer.h.33.attn.c_attn.qzeros', 'transformer.h.12.attn.c_attn.scales', 'transformer.h.28.attn.c_proj.qweight', 'transformer.h.27.attn.c_attn.g_idx', 'transformer.h.1.mlp.c_proj.scales', 'transformer.h.1.mlp.c_proj.g_idx', 'transformer.h.28.attn.c_proj.qzeros', 'transformer.h.33.mlp.c_fc.g_idx', 'transformer.h.29.attn.c_proj.scales', 'transformer.h.0.mlp.c_proj.qzeros', 'transformer.h.35.attn.c_proj.qweight', 'transformer.h.20.attn.c_attn.qzeros', 'transformer.h.32.mlp.c_fc.g_idx', 'transformer.h.26.mlp.c_proj.g_idx', 'transformer.h.4.mlp.c_fc.qzeros', 'transformer.h.16.mlp.c_fc.scales', 'transformer.h.37.mlp.c_proj.scales', 'transformer.h.14.attn.c_attn.qweight', 'transformer.h.33.mlp.c_proj.qweight', 'transformer.h.12.attn.c_proj.g_idx', 'transformer.h.2.mlp.c_proj.qzeros', 'transformer.h.4.attn.c_

In [6]:
for name, param in MODEL.named_parameters():
    print(name, param.numel())

transformer.wte.weight 257392640
transformer.wpe.weight 10485760
transformer.h.0.ln_1.weight 5120
transformer.h.0.ln_1.bias 5120
transformer.h.0.attn.c_attn.weight 19660800
transformer.h.0.attn.c_attn.bias 15360
transformer.h.0.attn.c_proj.weight 6553600
transformer.h.0.attn.c_proj.bias 5120
transformer.h.0.ln_2.weight 5120
transformer.h.0.ln_2.bias 5120
transformer.h.0.mlp.c_fc.weight 26214400
transformer.h.0.mlp.c_fc.bias 20480
transformer.h.0.mlp.c_proj.weight 26214400
transformer.h.0.mlp.c_proj.bias 5120
transformer.h.1.ln_1.weight 5120
transformer.h.1.ln_1.bias 5120
transformer.h.1.attn.c_attn.weight 19660800
transformer.h.1.attn.c_attn.bias 15360
transformer.h.1.attn.c_proj.weight 6553600
transformer.h.1.attn.c_proj.bias 5120
transformer.h.1.ln_2.weight 5120
transformer.h.1.ln_2.bias 5120
transformer.h.1.mlp.c_fc.weight 26214400
transformer.h.1.mlp.c_fc.bias 20480
transformer.h.1.mlp.c_proj.weight 26214400
transformer.h.1.mlp.c_proj.bias 5120
transformer.h.2.ln_1.weight 5120
tran

In [ ]:
CONFIG = IA3Config(
        peft_type="IA3",
        task_type="CAUSAL_LM",
        target_modules=["c_attn", "c_proj","c_fc"],
        feedforward_modules=['c_fc'])

MODEL = AutoModelForCausalLM.from_pretrained(BASE_MODEL, #load_in_8bit=True, 
                                             use_safetensors=True, device_map='cpu')

# frozing full base model
for param in MODEL.parameters():
    param.requires_grad = False

# wraps model to ia3
IA3_MODEL = get_peft_model(MODEL, CONFIG)

for name, param in IA3_MODEL.named_parameters():
    print(name, param.requires_grad)

# checking trainable parameters
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

print_trainable_parameters(IA3_MODEL)

In [9]:
input_ids = torch.tensor(np.random.randint(30, 50, size=(1,94)))

In [8]:
input_ids

tensor([[49., 49., 32., 32., 35., 40., 40., 36., 47., 35., 43., 42., 39., 46.,
         37., 32., 37., 32., 36., 32., 42., 33., 33., 45., 41., 41., 33., 30.,
         43., 48., 40., 32., 32., 39., 31., 32., 41., 49., 49., 32., 45., 47.,
         34., 40., 39., 45., 49., 31., 40., 32., 38., 40., 44., 37., 46., 32.,
         39., 47., 32., 41., 47., 32., 31., 45., 39., 39., 30., 37., 35., 35.,
         45., 30., 47., 32., 44., 46., 31., 41., 38., 36., 33., 38., 48., 40.,
         43., 39., 34., 41., 36., 39., 48., 48., 36., 35.]],
       dtype=torch.float16)

In [11]:
MODEL(input_ids=input_ids)

RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)

In [ ]:
OPTIMIZER = torch.optim.Adam(IA3_MODEL.parameters(), 
                             lr=LR)

In [ ]:
def train_one(model, loader, optimizer):
    """Standard PyTorch training, one epoch"""
    model.train()
    acc_loss, counter = [], 1
    process = tqdm.tqdm(loader)
    for batch in process:
        for k, v in batch.items():
            batch[k] = v.to(DEVICE)
        optimizer.zero_grad()
        out = model(input_ids=batch['input_ids'],
                    attention_mask=batch['attention_mask'],
                    labels=batch['labels'])
        
        loss = out['loss']
        acc_loss += loss.item()
        show_dict = {'Train cur mean Loss': f'{acc_loss/counter:.6f}'}
        process.set_postfix(show_dict)

    return round(acc_loss/counter, 5)

def eval_one(model, loader):
    """Standard PyTorch evaluation, one epoch"""
    model.eval()
    acc_loss, counter = [], 1
    process = tqdm.tqdm(loader)
    for batch in process:
        for k, v in batch.items():
            batch[k] = v.to(DEVICE)
            
        with torch.no_grad():
            out = model(input_ids=batch['input_ids'],
                        attention_mask=batch['attention_mask'],
                        labels=batch['labels'])
            
        loss = out['loss']
        acc_loss += loss.item()
        show_dict = {'Val cur mean Loss': f'{acc_loss/counter:.6f}'}
        process.set_postfix(show_dict)

    return round(acc_loss/counter, 5)

In [33]:
class PoemsDataset(Dataset):
    def __init__(self, file, tokenizer):

        self.df = pd.read_csv(file, sep=';') 
        self.tokenizer = tokenizer
        self.text_col = 'text'
        self.title_col = 'title'
        self.end_of_text_token = '<|endoftext|>'

    def __len__(self):
        return self.df.shape[0]
    
    def formate_sample(self, index):
        text, title = self.df[self.text_col][index], self.df[self.title_col][index] 
        
        formated_sttring = f'Название: "{title}". Стихотворение: {text} {self.end_of_text_token}'
        
        encoded_string = self.tokenizer(formated_sttring, return_tensors='pt', add_special_tokens=True)

        return {
            'input_ids': encoded_string['input_ids'],
            'attention_mask': encoded_string['atttention_mask'],
            'labels': encoded_string['input_ids']
        }

    def __getitem__(self, index):
        return self.formate_sample(index)

In [35]:
train_ds = PoemsDataset(TRAIN_FILE,TOKENIZER)
train_dataloader = DataLoader(train_ds, shuffle=True, batch_size=1)

eval_ds = PoemsDataset(EVAL_FILE,TOKENIZER)
eval_dataloader = DataLoader(eval_ds, shuffle=False, batch_size=1)

In [ ]:
best_eval_loss = None
for i_epoch in range(EPOCHS):
     loss_train = train_one(IA3_MODEL, train_dataloader, OPTIMIZER)
     loss_val = eval_one(IA3_MODEL, eval_dataloader)
     
     #
     print(f'{i_epoch} : loss_train={loss_train}, loss_val={loss_val}')
     metrics = {
          "epoch": i_epoch,
          "train_loss": loss_train, 
          "eval_loss": loss_val
          }
     
     # 
     if best_eval_loss is None or best_eval_loss >= loss_val:
          print("Saving new best AI3 adapter!")
          IA3_MODEL.save_pretrained(ADAPTER_DIR)
          best_eval_loss = loss_val
     elif i_epoch == (EPOCHS-1):
          print("Saving last AI3 adapter!")
          IA3_MODEL.save_pretrained(ADAPTER_DIR + '(last_train)')

     #
     m_object = json.dumps(metrics, indent=2, ensure_ascii=False)
     with open(LOG_DIR + f'epoch_{i_epoch}', 'w',encoding='utf-8') as fd:
          fd.write(m_object)

In [ ]:
# test